In [2]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load the data from your 'data' folder
# glob="*.md" ensures we only read markdown files
loader = DirectoryLoader(
    './data', 
    glob="*.md", 
    loader_cls=TextLoader
)
documents = loader.load()

print(f"✅ Loaded {len(documents)} documents (files).")

# 2. Split the data into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Characters per chunk
    chunk_overlap=50 # Overlap to keep context between chunks
)
chunks = text_splitter.split_documents(documents)

print(f"✅ Split into {len(chunks)} smaller chunks.")

# 3. Inspect the first chunk to verify
print("\n--- Sample Chunk ---")
print(chunks[0].page_content)
print("--------------------")

✅ Loaded 3 documents (files).
✅ Split into 27 smaller chunks.

--- Sample Chunk ---
# About Ankit Sharma

Born: January 8, 2005 (Age 21)  
Gender: Male  
Religion: Hindu  
Category: General  
From: VPO Karloti, Tehsil Ghumarwin, District Bilaspur, Himachal Pradesh 174029

Passionate B.Tech Computer Science student from VIT Bhopal University with strong academic record and professional certifications. Currently building expertise in AI/ML through research internships and self-driven projects. Cricket enthusiast, sports lover, and aspiring family provider.

---
--------------------


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# 1. Define the Embedding Model
# We use "all-MiniLM-L6-v2". It is the industry standard for fast, free, local embeddings.
print("⏳ Loading embedding model...")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Create the Vector Database
# This converts your 27 chunks into vectors.
print("⏳ Creating vector index from chunks...")
vector_store = FAISS.from_documents(chunks, embedding_model)

# 3. Save it to disk
# This allows us to load it later without re-running this code.
vector_store.save_local("faiss_index")

print("✅ Success! Vector Store saved to folder: 'faiss_index'")

⏳ Loading embedding model...
⏳ Creating vector index from chunks...
✅ Success! Vector Store saved to folder: 'faiss_index'


In [4]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# 1. Load the Embedding Model (Same one used to create the index)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 2. Load your saved Vector Store
# allow_dangerous_deserialization=True is required for local files (safe here since you created them)
print("⏳ Loading Vector Store...")
vector_store = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)

# 3. Run a Test Search
query = "Tell me about the Brain Tumor Detection project"
# k=3 means "Give me the top 3 most relevant chunks"
results = vector_store.similarity_search(query, k=3)

print(f"\n🔍 Query: {query}")
print("-" * 40)
for i, doc in enumerate(results):
    print(f"📄 Result {i+1} (Source: {doc.metadata['source']}):")
    print(doc.page_content[:300] + "...")  # Show first 300 chars
    print("-" * 40)

⏳ Loading Vector Store...

🔍 Query: Tell me about the Brain Tumor Detection project
----------------------------------------
📄 Result 1 (Source: data\project.md):
# Computer Vision & Medical AI

## Brain Tumor Detection System (Featured Project)

Multi-class MRI tumor classification with 94%+ accuracy using CNNs.

**Problem:** Classify brain MRI images into glioma, meningioma, pituitary, and no tumor.

**Dataset:** 3,000+ T1-weighted MRI scans with augmentati...
----------------------------------------
📄 Result 2 (Source: data\project.md):
**Architecture:**

- Custom CNN: Conv2D â†’ MaxPool â†’ Dropout â†’ Dense layers
- Transfer Learning: VGG16 / ResNet50 fine-tuned
- Preprocessing: Resize 150x150, normalization, CLAHE enhancement

**Performance Metrics:**

- Test Accuracy: 94.23% (1324/1405 images)
- Validation Accuracy: 91â€“94%
- ...
----------------------------------------
📄 Result 3 (Source: data\project.md):
**Tech Stack:** PyTorch, OpenCV, CASIA-B preprocessing scripts.

Status

In [9]:
import os
from getpass import getpass
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from huggingface_hub import InferenceClient # <--- The Official Client

# 1. Setup API Key
if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = getpass("🔑 Enter your Hugging Face Token: ")

# 2. Reload Vector Store
print("⏳ Reloading Vector Store...")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 3. Define the Chat Function (The "Fix")
# We use the official client to talk to Zephyr-7B
def query_zephyr_chat(prompt_text):
    client = InferenceClient(api_key=os.environ["HF_TOKEN"])
    
    # We construct a simple "User" message
    messages = [
        {"role": "user", "content": prompt_text}
    ]
    
    try:
        # We use 'chat_completion' which is the modern standard for these models
        response = client.chat_completion(
            model="HuggingFaceH4/zephyr-7b-beta",
            messages=messages,
            max_tokens=500,
            temperature=0.5
        )
        # Extract the answer text
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# 4. Define Prompt
template = """You are Ankit's AI Assistant. Answer strictly based on the context.
If you don't know, say "I don't have that info. and keep the answer in fun way , in chill moood"

Context:
{context}

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# 5. Build the Chain
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | (lambda x: x.to_string())  # Convert Prompt to String
    | query_zephyr_chat          # Send to Zephyr Chat
)

print("✅ SUCCESS! Chain built with InferenceClient.")

# 6. Test Immediately
print("🤖 Testing...")
response = rag_chain.invoke("what is ankit sister name ?")
print("\n" + "="*40)
print(response)
print("="*40)

⏳ Reloading Vector Store...
✅ SUCCESS! Chain built with InferenceClient.
🤖 Testing...

I'm sorry but the provided context doesn't include any information about Ankit's sister's name. I'd say, "I don't have that info. And let's keep the answer in a chill moood by suggesting that Ankit seems to be quite focused on his academic, professional, and personal pursuits, but we're not privy to his sister's name at this time. Maybe you can ask him directly or do some sleuthing on social media to uncover that secret detail, but for now, let's just focus on celebrating his impressive accomplishments and talents in sports, music, and cooking! 🍳🎧🏃‍♂️🏋️‍♂️🎤🏀🍳 #ankitsharma1831ao #chillvibes #passionatelearner #selfdriven #himachalipride #techwhizkid #chefankitsharma" 😎🤩🤩🤩🤩 #keepcalmandcookon #fitnessfun #githubguru
